# Cryptocurrency data analysis and price prediction using ARIMA and Seasonal ARIMA model
#### Project Proposal

## 1. Aims, objective and background
### 1.1 Introduction
Cryptocurrencies are quickly challenging traditional currencies throughout the world. Digital currencies may be purchased in a variety of ways, making them accessible to everyone, and merchants adopting multiple cryptocurrencies could signal that money as we know it is about to undergo a huge transformation.

Bitcoin was the first cryptocurrency launched in the year 2009 and since then the popularity and acceptibility of blockchain and cryptocurrencies has only grown upward direction. Due to its unique qualities of blockchain, such as security and transparency, cheaper cost and decentralisation, it is already being used to solve variety of real world problems as per Mallqui and Fernandes [5].

This project was inspired by Chaudhari, A. (2020, June 11). Forecasting Cryptocurrency Prices using Machine Learning [3] and Chakrabarti, S. (2021, December 3). Cryptocurrency Price Prediction using ARIMA Model [4]. The authors used data science ARIMA, LSTM and Prophet models to predict the cryptocurrency prices.

Currently the interest of investing in crypto is growning rapidly and there is less information about crypto relative to our traditional trading and this is a new platform for me researching. I love to research new things. So, I have decided to focus on next few months price prediction of top 4 cryptocurrencies.

### 1.2 Aims and objectives
For this project I would like to exlpore following things
1. Analysing of history data of cryptocurrency.
2. Using history data making visualizations (graphs)
3. Looking for relationship of the graphs.
4. Checking the instantaneous move of one coin (BTC) affect others.
5. Using (Autoregressive integrated moving average) model and SARIMAX (Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors) model for predicting next months prices.

For this project my aims are
1. Decide how much history data is required to use ARIMA and SARIMAX models.
2. Decide which currencies will be best for my project.
3. Collecting data via API/wesrapping and storing in better format so that I can do data cleaning.
4. Clean the scraped or API history data suitable for data analysis
5. Perform some data analysis to see if there are any trends in the data that may be helpful for future investigation.
6. Calculate the accuracy of price predication.

### 1.3 Data
#### 1.3.1 Data requirement
Right now there are more than 6000 crypto coins in the market. Taking consideration of time and resource. I have chosen to work on top 4 crypto coins to explore.
I have decided to have equal number of history data for the coins and going to use data from one website because there are lot of exchanges and each exchange have slight differences.

#### 1.3.2 Choice of history data website
For the history data of coins, I am going to choose 1 website but first I made a list of good sources.

1) www.investing.com        Scrapping / No API
2) www.coinmarketcap.com    No Scrapping / API
3) www.coingecko.com        No Scrapping / API
4) www.coinpaprika.com      No Scrapping / API

In this list only www.investing.com allowed data scrapping and other websites allowed on APIs only. API will be easy to collect data but then I will not able to use my new ability to scrap data. So, I am choosing www.investing.com [1].

#### 1.3.3 Choice of the cryptocurrencies: methodology
First point for choosing the currency will be the enough history data for analysis. Keeping the coin not related to each other. After it we need to check their rank [7], daily volume.

Following are the selected coins:
1) Bitcoin (BTC)
2) Ethereum (ETH)
3) Cardano (ADA)
4) XRP (XRP)

Originally I have choosen top 4 currencies but some don't have enough history for analysis and some are related to other currencies. They may have affected each other with the price change. So, I have skipped Binance Coin (BNB) and Solana (SOL).

#### 1.3.4 Limitations and constraints of the data
In this project I am going to use last 3 years of history data with per day value for analyase. This means I not going to use data from when the coin initiated in the market and this data not going to be per second/minute it will be per day.

For analysing large data (per second or per minutes) will cause lot longer time for scrapping, cleaning and analysing the data. So, taking the account of limited time and resources I am going to use per day history values.

Everyone know there are different event happened and it may have caused the price fluctuations, for example last few days everyone knows COVID caused lot of sudden fall in all the markets.

### 1.4 Ethical considerations
#### 1.4.1 Use of cryptocurrency history data
www.investing.com allows to use their data but only ask to make sure to include full disclosure to Investing.com brand, logo, watermark and links if possible [1].

#### 1.4.2 Onward use / reuseage and derived data
Anyone wishing to use the source data must follow the term and conditions of www.investing.com and should take indiidual permission if necessary. The same term and condition apply to data producted from source data.

Analysis and conclusions are my own.

#### 1.4.3 Potential impacts of using cryptocurrency data for the proposes analyses
Doing analysis of the source data may cause negative effect on the traders/cryptocurrencies. This may potential losses.
1. Rather than making judgments on the currency will grow or fall. This project only focus on analysing objective.
2. The project's findings will not claim to be a fully representative analysis. Limitations of data are outlined above. Limitations of techniques will be discussed further project.



In [5]:
# Importing libraries and modules
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
import seaborn as sns

import csv
from datetime import date
import datetime

# Show all matplotlib graphs inline
%matplotlib inline

# Set all graphs to a seaborn style with a grey background grid which makes reading graphs easier
sns.set()

## 2. Web Scraping cryptocurrencies history data
### 2.1 Defining scraping and extraction functions
There going to be 3 years data for each coin.

1) Bitcoin (BTC)
2) Ethereum (ETH)
3) Cardano (ADA)
4) XRP (XRP)

Before importing the data we need to make a function to iterate over the 4 urls.
All the data of each coin going to be on 1 page so, data scraping will take few seconds. So, we don't need to worry about getting blocked or blacklisted.

Below function will check if the website is accessible and returns the content.

In [23]:
today = date.today()
today = date.strftime(today, '%d/%m/%Y')

# coinInfo dataframe includes required values to scrap data from Investing.com
coinInfo = { 'bitcoin': {'name': 'bitcoin', 'symbol': 'BTC', 'curr_id': 1057391, 'smlID': 25609848},
            'ethereum': {'name': 'ethereum', 'symbol': 'ETH', 'curr_id': 1061443, 'smlID': 25674078},
            'cardano': {'name': 'cardano', 'symbol': 'ADA', 'curr_id': 1062537, 'smlID': 25948924},
            'xrp': {'name': 'xrp', 'symbol': 'XRP', 'curr_id': 1057392, 'smlID': 25674343},}

# getParsedWebPage function will scrap data from Investing.com and convert it to dataframe
# Date format should in 'DD/MM/YYYY"
def getParsedWebPage(coinName, from_date, to_date):

    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36", "X-Requested-With": "XMLHttpRequest"}

    url = "https://in.investing.com/instruments/HistoricalDataAjax"
    header = 'null'

    payload = {'header': header,
               'st_date': from_date, 'end_date': to_date,
               'sort_col': 'date', 'action': 'historical_data',
               'smlID': coinInfo[coinName], 'sort_ord': 'DESC', 'interval_sec': 'Daily', 'curr_id': coinInfo[coinName]['curr_id']}

    res = requests.post(url, headers=headers, data=payload)

    # Check that page is accessible for scraping
    if res.status_code != 200:
        soup = 'error'
    else:
        soup = BeautifulSoup(res.content, "lxml")
        table = soup.find('table', id="curr_table")
        df = pd.read_html(str(table))[0]

    # Adding Coin symbol and rearranging the columns
    df['Symbol'] = 'XRP'
    df.rename(columns={'Vol.': 'Volume'}, inplace=True)
    df = df[['Date', 'Symbol', 'Price', 'Open', 'High', 'Low', 'Volume']]
    return df

,Date,Symbol,Price,Open,High,Low,Volume
0,"Dec 12, 2021",XRP,0.83990,0.83767,0.85633,0.80961,441.73M
1,"Dec 11, 2021",XRP,0.83722,0.79999,0.84336,0.78641,561.08M
2,"Dec 10, 2021",XRP,0.79916,0.85795,0.88198,0.79526,957.23M
3,"Dec 09, 2021",XRP,0.85809,0.86167,0.93300,0.83285,1.26B
4,"Dec 08, 2021",XRP,0.86148,0.81535,0.88302,0.80486,811.57M
...,...,...,...,...,...,...,...
361,"Dec 16, 2020",XRP,0.57120,0.46777,0.57337,0.43985,2.50B
362,"Dec 15, 2020",XRP,0.46783,0.49735,0.50706,0.46489,1.26B
363,"Dec 14, 2020",XRP,0.49747,0.51392,0.51810,0.48986,869.69M
364,"Dec 13, 2020",XRP,0.51416,0.50589,0.52755,0.48652,1.28B


## 7. References and Resources
### 7.1 References
[1]: Use our data. (n.d.). Investing Support. https://www.investing-support.com/hc/en-us/articles/360002357417
[2]: FRANKENFIELD, J. A. K. E. (2021, December 20). Cryptocurrency. Investopedia. https://www.investopedia.com/terms/c/cryptocurrency.asp
[3]: Chaudhari, A. (2020, June 11). Forecasting Cryptocurrency Prices using Machine Learning - NORMA@NCI Library. Norma.Ncirl.Ie. http://norma.ncirl.ie/4272/
[4]: Chakrabarti, S. (2021, December 3). Cryptocurrency Price Prediction using ARIMA Model. Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/12/cryptocurrency-price-prediction-using-arima-model/
[5]: Mallqui, D., & Fernandes, R. (2019b, February 1). Predicting the direction, maximum, minimum and closing prices of daily Bitcoin exchange rate using machine learning techniques. ScienceDirect. https://www.sciencedirect.com/science/article/pii/S1568494618306707
[6]: George, D. (2021, December 14). A Brief Introduction to ARIMA and SARIMAX Modeling in Python. Medium. https://medium.com/swlh/a-brief-introduction-to-arima-and-sarima-modeling-in-python-87a58d375def.

[7]: CoinMarketCap. (n.d.-b). Today’s Top 100 Crypto Coins Prices And Data. Retrieved December 26, 2021, from https://coinmarketcap.com/coins/

[8]: Rai, A. (2021, December 11). Python utility for data scrapping historical financial data- ML Data mining. Medium. https://medium.datadriveninvestor.com/python-utility-for-data-scrapping-historical-financial-data-ml-data-mining-5396dfe6f38c


### 7.2 Resources used